# Epic 8
Als een keyuser wil ik graag een lijst van accounts (bedrijven) kunnen opmaken die het meest waarschijnlijk zijn om hun lidmaatschap volgend jaar niet te hernieuwen

Om dit te kunnen uitvoeren, gaan we eerst inzicht in de data omtrent lidmaatschappen verwerven. Het is vooral belangrijk te achterhalen om welke redenen accounts in het verleden hun lidmaatschap hebben opgezegd.

## 1. Data inladen

In [661]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import optparse
import nltk
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
import seaborn as sns

#### Create connection to DWH


nog geen OS gebruikt hieronder want gebruik docker dus moet nog es bekijken hoe

In [662]:

SERVER_NAME_REMOTE="localhost"
DB_USER="sa"
DB_PASSWORD="Dep2Groep2-VIC"
DWH_NAME="Voka_DWH"

def connect_db(local=True):
    if local:
        URL_LOCAL = f'mssql+pyodbc://{SERVER_NAME}/{DWH_NAME}?trusted_connection=yes&driver=ODBC+Driver+17 for SQL Server'
        engine = create_engine(URL_LOCAL)
        conn = engine.connect()
        return conn
    else:
        URL = f'mssql+pymssql://{DB_USER}:{DB_PASSWORD}@{SERVER_NAME_REMOTE}:1438/{DWH_NAME}'
        engine = create_engine(URL)
        conn = engine.connect()
        return conn

In [663]:
conn = connect_db(local=False)

Create dataframe with all the data from the DWH

In [664]:
def create_query(table_name, columns, condition=None):

    query = f"SELECT "

    for i, column in enumerate(columns):
        if i == 0:
            query += f"[{column}]"
        else:
            query += f", [{column}]"
    
    query += f" FROM [{DWH_NAME}].[dbo].[{table_name}]"
    
    if condition:
        query += f" WHERE {condition}"

    return query


#### Merge Account en Lidmaatschap

In [665]:
#ACCOUNT DATA
#needed Account columns
account_col = ['accountID', 'plaats', 'isVokaEntiteit', 'accountStatus', 'ondernemingstype', 'ondernemingsaard', 'activiteitNaam']

#condition
account_condition = "provincie = 'Oost-Vlaanderen'"

#create query
account_query = create_query('DimAccount', account_col, account_condition)
df_account = pd.read_sql(account_query, conn)
df_account['plaats'] = df_account['plaats'].str.replace(r'\([a-z.-]+\)', '', regex=True).str.replace('  ', ' ')


df_account.shape


(50170, 7)

In [666]:
#LIDMAATSCHAP DATA
#needed Lidmaatschap columns
lidmaatschap_col = ['lidmaatschapID', 'accountID', 'redenAangroei', 'redenVerloop', 'startDatum', 'opzegDatum']

#create query
lidmaatschap_query = create_query('DimLidmaatschap', lidmaatschap_col)
df_lidmaatschap = pd.read_sql(lidmaatschap_query, conn)

df_lidmaatschap.shape

(6612, 6)

In [667]:
#FIN DATA
#needed Fin columns
financiële_data_col = ['accountID', 'boekjaar', 'aantalMaanden', 'toegevoegdeWaarde']

#create query
financiële_data_query = create_query('DimFinanciëleDataAccount', financiële_data_col)
df_financiële_data = pd.read_sql(financiële_data_query, conn)

df_financiële_data.shape

(317954, 4)

In [668]:
#Inschrijving Data
#needed Inschrijving columns
inschrijving_col = ['contactID', 'campagneID', 'inschrijvingsDatumID', 'facturatieBedrag']

#create query
inschrijving_query = create_query('FactInschrijving', inschrijving_col)
df_inschrijving = pd.read_sql(inschrijving_query, conn)

df_inschrijving.shape

(78790, 4)

#### Merge Lidmaatschap en Account

In [669]:
df_account_lid = df_account.merge(df_lidmaatschap, on='accountID', how='inner')
df_account_lid.shape

(6612, 12)

#### Merge Inschrijving en Contact

In [670]:
#needed contact col
contact_col = ['contactID', 'accountID']

#create query
contact_query = create_query('DimContact', contact_col)
df_contact = pd.read_sql(contact_query, conn)

df_contact.shape

(194192, 2)

In [671]:
df_inschrijving = df_inschrijving.merge(df_contact, on='contactID', how='left')
df_inschrijving.shape

(78790, 5)

#### Merge Inschrijving en Datum

In [672]:
#needed datum col
datum_col = ['dateID', 'fullDate']

#create query
datum_query = create_query('DimDate', datum_col)
df_datum = pd.read_sql(datum_query, conn)

df_datum.shape

(100806, 2)

In [673]:
df_inschrijving = df_inschrijving.merge(df_datum, left_on='inschrijvingsDatumID', right_on='dateID', how='left')


In [674]:
df_inschrijving.drop(columns=['dateID', 'inschrijvingsDatumID'], inplace=True)

In [675]:
df_inschrijving.head()

,contactID,campagneID,facturatieBedrag,accountID,fullDate
0,0017416A-2C6E-E111-B43A-00505680000A,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21
1,0017416A-2C6E-E111-B43A-00505680000A,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21
2,0017416A-2C6E-E111-B43A-00505680000A,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21
3,0017416A-2C6E-E111-B43A-00505680000A,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21
4,0017416A-2C6E-E111-B43A-00505680000A,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21


In [676]:
#drop contactID from inschrijving
df_inschrijving.drop('contactID', axis=1, inplace=True)
df_inschrijving.shape

(78790, 4)

#### Inladen Campagne 

In [677]:
#needed campagne col
campagne_col = ['campagneID', 'campagneNaam', 'campagneType']

#create query
campagne_query = create_query('DimCampagne', campagne_col)
df_campagne = pd.read_sql(campagne_query, conn)

df_campagne.shape

(4101, 3)

We hebben nu de volgende dataframes:
- df_account_lid = account + lidmaatschap info
- df_inschrijving = inschrijvingen per account
- df_financiële_data = financiële data per account
- df_campagne = de verschillende campagnes waarvoor een account kan inschrijven

## 2. Data analyseren

In [678]:
df_account_lid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6612 entries, 0 to 6611
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   accountID         6612 non-null   object
 1   plaats            6612 non-null   object
 2   isVokaEntiteit    6612 non-null   int64 
 3   accountStatus     6612 non-null   int64 
 4   ondernemingstype  6612 non-null   object
 5   ondernemingsaard  6612 non-null   object
 6   activiteitNaam    2448 non-null   object
 7   lidmaatschapID    6612 non-null   object
 8   redenAangroei     6612 non-null   object
 9   redenVerloop      6612 non-null   object
 10  startDatum        6612 non-null   object
 11  opzegDatum        6612 non-null   object
dtypes: int64(2), object(10)
memory usage: 620.0+ KB


In [679]:
df_account_lid.head()

,accountID,plaats,isVokaEntiteit,accountStatus,ondernemingstype,ondernemingsaard,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum
0,00002DAC-0A69-E111-B43A-00505680000A,BEVERE,0,1,Bedrijf,Diensten,Overige industrie & diensten,F10DF4A4-186F-E111-B43A-00505680000A,unknown,unknown,2011-12-07,2013-12-31
1,00025050-8D55-EB11-8117-001DD8B72B61,Melle,0,1,Familiebedrijf,Diensten,None,EB02B961-EFC1-EB11-8123-001DD8B72B62,Lead van een collega nalv contact of deelname,unknown,2021-05-20,2026-01-01
2,0012D444-0A69-E111-B43A-00505680000A,GERAARDSBERGEN,0,1,Bedrijf,Diensten,None,DFECE67E-146F-E111-B43A-00505680000A,unknown,Geen gebruik,2012-03-02,2016-03-28
3,0014BCC0-075B-EB11-811A-001DD8B72B62,MEILEGEM,0,1,Bedrijf,Diensten,None,42A999FB-6F69-ED11-9561-6045BD8952CE,Actieve werving: koude prospectie,unknown,2022-11-18,2026-01-01
4,0016CAE8-BD68-E111-B43A-00505680000A,GENTBRUGGE,0,1,Bedrijf,Productie & Diensten,Overige industrie & diensten,36CF5725-FA6E-E111-B43A-00505680000A,unknown,unknown,2010-12-17,2026-01-01


We gaan nu kijken naar de accounts waarvoor de accountStatus = 0. Dit zijn accounts die hun lidmaatschap hebben opgezegd. We willen nu achterhalen waarom ze dit hebben gedaan.

De meeste info die we hier voorlopig uit kunnen halen is voorlopig bij de reden 'Geen gebruik'. 'Unknown' is namelijk niet bruikbaar en 'Stopzetting' is te vaag. We gaan dus kijken naar de accounts waarvoor de reden 'Geen gebruik' is. (En we verwijderen de accounts waarvoor de reden 'Unknown' is => we maken nieuwe dataframe)

In [680]:
print(df_account_lid['opzegDatum'].unique())

[datetime.date(2013, 12, 31) datetime.date(2026, 1, 1)
 datetime.date(2016, 3, 28) ... datetime.date(2016, 3, 17)
 datetime.date(2012, 10, 25) datetime.date(2014, 4, 22)]


In [681]:
#opzegdatum omzetten naar string
df_account_lid['opzegDatum'] = df_account_lid['opzegDatum'].astype(str)
#we splitsen de dataframes in accounts waarvoor de opzegtdatum = 1950-01-01 en accounts waarvoor de opzegdatum != 1950-01-01
df_account_ACTIEF = df_account_lid[df_account_lid['opzegDatum'] == '2026-01-01']
df_account_INACTIEF = df_account_lid[df_account_lid['opzegDatum'] != '2026-01-01']

df_account_INACTIEF.shape


(4060, 12)

In [682]:
# #we verwijderen de rijen waarvoor redenVerloop = "unkown"
# df_account_INACTIEF = df_account_INACTIEF[df_account_INACTIEF['redenVerloop'] != 'unknown']
# df_account_INACTIEF.shape

In [683]:
#nagaan voor accounts met accountStatus = 0, wat redenVerloop is + sinds wanneer
df_account_INACTIEF['redenVerloop'].value_counts()


redenVerloop
unknown                      1841
Geen gebruik                 1007
Stopzetting                   409
Overzetting lidmaatschap      312
Wanbetaler                    139
Geen meerwaarde                90
Ontevreden/klacht              84
Prijs                          82
Faillissement                  75
Overname                       17
FinanciÃ«le moeilijkheden       4
Name: count, dtype: int64

In [684]:
df_account_ACTIEF.head(10)


,accountID,plaats,isVokaEntiteit,accountStatus,ondernemingstype,ondernemingsaard,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum
1,00025050-8D55-EB11-8117-001DD8B72B61,Melle,0,1,Familiebedrijf,Diensten,None,EB02B961-EFC1-EB11-8123-001DD8B72B62,Lead van een collega nalv contact of deelname,unknown,2021-05-20,2026-01-01
3,0014BCC0-075B-EB11-811A-001DD8B72B62,MEILEGEM,0,1,Bedrijf,Diensten,None,42A999FB-6F69-ED11-9561-6045BD8952CE,Actieve werving: koude prospectie,unknown,2022-11-18,2026-01-01
4,0016CAE8-BD68-E111-B43A-00505680000A,GENTBRUGGE,0,1,Bedrijf,Productie & Diensten,Overige industrie & diensten,36CF5725-FA6E-E111-B43A-00505680000A,unknown,unknown,2010-12-17,2026-01-01
5,002304C8-C268-E111-B43A-00505680000A,BUGGENHOUT,0,1,Familiebedrijf,Productie,None,2ACEEBEE-126F-E111-B43A-00505680000A,unknown,unknown,2007-01-01,2026-01-01
6,002C0FBC-C268-E111-B43A-00505680000A,Hamme (O.-Vl.),0,1,Familiebedrijf,Productie & Diensten,Ijzer en staal,A6713FE6-FB6E-E111-B43A-00505680000A,unknown,unknown,2007-01-01,2026-01-01
8,003EB9D8-0A29-E811-80F0-001DD8B72B61,Ledeberg,0,1,Bedrijf,Diensten,None,CF9E5EAE-41C6-E811-80F5-001DD8B72B62,Actieve werving: koude prospectie,unknown,2018-10-01,2026-01-01
9,004CB1A1-4C92-ED11-AAD1-6045BD895D85,Gent,0,1,Bedrijf,Diensten,None,C6A46201-4E92-ED11-AAD1-6045BD895D85,unknown,unknown,2023-01-09,2026-01-01
10,004E24A6-0A69-E111-B43A-00505680000A,DENDERMONDE,0,1,Bedrijf,Diensten,Telecom & IT,CF25E5EF-E76E-E111-B43A-00505680000A,unknown,unknown,2012-01-17,2026-01-01
11,0059AC0E-2D17-EA11-8109-001DD8B72B62,SINT-MARTENS-LATEM,0,1,Bedrijf,Diensten,None,9F9DF7C6-2D17-EA11-8109-001DD8B72B62,Actieve werving: koude prospectie,unknown,2019-11-30,2026-01-01
15,006A3322-F65B-E711-80E8-001DD8B72B61,DENDERMONDE,0,1,Bedrijf,Productie,None,B338E25F-F65B-E711-80E8-001DD8B72B61,unknown,Overzetting lidmaatschap,2007-01-01,2026-01-01


Nu willen we voor een bepaald accountID uit de dataframe df_account_INACTIEF de aantal inschrijvingen tellen (per campagne) per jaar. We willen dit doen voor telkens het jaar VOOR het jaar van de opzegDatum.

In [685]:
#jaar voor opzegDatum per acountID bepalen
df_account_INACTIEF['jaar'] = df_account_INACTIEF['opzegDatum'].str[:4] 
df_account_INACTIEF['voorbije_jaar'] = df_account_INACTIEF['jaar'].astype(int) - 1
df_account_INACTIEF.drop('jaar', axis=1, inplace=True)

df_account_INACTIEF.head()

C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3134376907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_INACTIEF['jaar'] = df_account_INACTIEF['opzegDatum'].str[:4]
C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3134376907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_INACTIEF['voorbije_jaar'] = df_account_INACTIEF['jaar'].astype(int) - 1
C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3134376907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,accountID,plaats,isVokaEntiteit,accountStatus,ondernemingstype,ondernemingsaard,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum,voorbije_jaar
0,00002DAC-0A69-E111-B43A-00505680000A,BEVERE,0,1,Bedrijf,Diensten,Overige industrie & diensten,F10DF4A4-186F-E111-B43A-00505680000A,unknown,unknown,2011-12-07,2013-12-31,2012
2,0012D444-0A69-E111-B43A-00505680000A,GERAARDSBERGEN,0,1,Bedrijf,Diensten,None,DFECE67E-146F-E111-B43A-00505680000A,unknown,Geen gebruik,2012-03-02,2016-03-28,2015
7,003B965B-C268-E111-B43A-00505680000A,GENT,0,1,Bedrijf,Diensten,Telecom & IT,D7D751D6-2D6F-E111-B43A-00505680000A,Actieve werving: koude prospectie,Geen gebruik,2013-07-04,2017-01-19,2016
12,005C6F7F-A268-E111-B43A-00505680000A,SINT-MARTENS-LATEM,0,1,Bedrijf,Diensten,None,4F8AA590-BA47-EC11-8C62-6045BD8D2834,Lead van een collega nalv contact of deelname,Prijs,2021-11-16,2021-12-30,2020
13,0062F706-0969-E111-B43A-00505680000A,DEINZE,0,1,Bedrijf,Productie,None,45C679CB-086F-E111-B43A-00505680000A,unknown,Geen gebruik,2007-01-02,2013-07-01,2012


Nu hebben we dus de dataframe van accounts die hun lidmaatschap hebben opgezegd bij Voka + het jaar VOOR de opzegdatum van deze accounts. Nu gaan we voor dit jaar gaan tellen hoeveel inschrijvingen er zijn per accountID per campagne.

In [686]:
#voorbereiding inschrijving
df_inschrijving['fullDate'] = df_inschrijving['fullDate'].astype(str)
df_inschrijving['jaar'] = df_inschrijving['fullDate'].str[:4]
df_inschrijving['jaar'] = df_inschrijving['jaar'].astype(int)
df_inschrijving.head()


,campagneID,facturatieBedrag,accountID,fullDate,jaar
0,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21,2019
1,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21,2019
2,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21,2019
3,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21,2019
4,82D778E0-1EC3-E811-80F5-001DD8B72B62,3000,18C73253-C368-E111-B43A-00505680000A,2019-01-21,2019


In [687]:
#hoeveel inschrijvingen per accountID van df_account_INACTIEF in het voorbije jaar (zie kolom voorbije_jaar in df) per campagneID
for i, row in df_account_INACTIEF.iterrows():
    accountID = row['accountID']
    voorbije_jaar = row['voorbije_jaar']
    df_account_INACTIEF.loc[i, 'aantal_inschrijvingen'] = df_inschrijving[(df_inschrijving['accountID'] == accountID) & (df_inschrijving['jaar'] == voorbije_jaar)]['accountID'].count()

C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\4123443639.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_INACTIEF.loc[i, 'aantal_inschrijvingen'] = df_inschrijving[(df_inschrijving['accountID'] == accountID) & (df_inschrijving['jaar'] == voorbije_jaar)]['accountID'].count()


In [688]:
df_account_INACTIEF.head()

,accountID,plaats,isVokaEntiteit,accountStatus,ondernemingstype,ondernemingsaard,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum,voorbije_jaar,aantal_inschrijvingen
0,00002DAC-0A69-E111-B43A-00505680000A,BEVERE,0,1,Bedrijf,Diensten,Overige industrie & diensten,F10DF4A4-186F-E111-B43A-00505680000A,unknown,unknown,2011-12-07,2013-12-31,2012,0.0
2,0012D444-0A69-E111-B43A-00505680000A,GERAARDSBERGEN,0,1,Bedrijf,Diensten,None,DFECE67E-146F-E111-B43A-00505680000A,unknown,Geen gebruik,2012-03-02,2016-03-28,2015,0.0
7,003B965B-C268-E111-B43A-00505680000A,GENT,0,1,Bedrijf,Diensten,Telecom & IT,D7D751D6-2D6F-E111-B43A-00505680000A,Actieve werving: koude prospectie,Geen gebruik,2013-07-04,2017-01-19,2016,0.0
12,005C6F7F-A268-E111-B43A-00505680000A,SINT-MARTENS-LATEM,0,1,Bedrijf,Diensten,None,4F8AA590-BA47-EC11-8C62-6045BD8D2834,Lead van een collega nalv contact of deelname,Prijs,2021-11-16,2021-12-30,2020,0.0
13,0062F706-0969-E111-B43A-00505680000A,DEINZE,0,1,Bedrijf,Productie,None,45C679CB-086F-E111-B43A-00505680000A,unknown,Geen gebruik,2007-01-02,2013-07-01,2012,0.0


In [689]:
# voor hoeveel accounts is aantal inschrijvingen = 0
df_account_INACTIEF[df_account_INACTIEF['aantal_inschrijvingen'] == 0]['accountID'].count()

3671

In [690]:
df_account_ACTIEF['voorbije_jaar'] = 2022
df_account_ACTIEF.shape

C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3430865321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_ACTIEF['voorbije_jaar'] = 2022


(2552, 13)

In [691]:
for i, row in df_account_ACTIEF.iterrows():
    accountID = row['accountID']
    voorbije_jaar = row['voorbije_jaar']
    df_account_ACTIEF.loc[i, 'aantal_inschrijvingen'] = df_inschrijving[(df_inschrijving['accountID'] == accountID) & (df_inschrijving['jaar'] == voorbije_jaar)]['accountID'].count()

C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\20846879.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_ACTIEF.loc[i, 'aantal_inschrijvingen'] = df_inschrijving[(df_inschrijving['accountID'] == accountID) & (df_inschrijving['jaar'] == voorbije_jaar)]['accountID'].count()


In [692]:
df_account_ACTIEF.head()

,accountID,plaats,isVokaEntiteit,accountStatus,ondernemingstype,ondernemingsaard,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum,voorbije_jaar,aantal_inschrijvingen
1,00025050-8D55-EB11-8117-001DD8B72B61,Melle,0,1,Familiebedrijf,Diensten,None,EB02B961-EFC1-EB11-8123-001DD8B72B62,Lead van een collega nalv contact of deelname,unknown,2021-05-20,2026-01-01,2022,0.0
3,0014BCC0-075B-EB11-811A-001DD8B72B62,MEILEGEM,0,1,Bedrijf,Diensten,None,42A999FB-6F69-ED11-9561-6045BD8952CE,Actieve werving: koude prospectie,unknown,2022-11-18,2026-01-01,2022,5.0
4,0016CAE8-BD68-E111-B43A-00505680000A,GENTBRUGGE,0,1,Bedrijf,Productie & Diensten,Overige industrie & diensten,36CF5725-FA6E-E111-B43A-00505680000A,unknown,unknown,2010-12-17,2026-01-01,2022,0.0
5,002304C8-C268-E111-B43A-00505680000A,BUGGENHOUT,0,1,Familiebedrijf,Productie,None,2ACEEBEE-126F-E111-B43A-00505680000A,unknown,unknown,2007-01-01,2026-01-01,2022,0.0
6,002C0FBC-C268-E111-B43A-00505680000A,Hamme (O.-Vl.),0,1,Familiebedrijf,Productie & Diensten,Ijzer en staal,A6713FE6-FB6E-E111-B43A-00505680000A,unknown,unknown,2007-01-01,2026-01-01,2022,16.0


In [693]:
# voor hoeveel accounts is aantal inschrijvingen = 0
df_account_ACTIEF[df_account_ACTIEF['aantal_inschrijvingen'] == 0]['accountID'].count()

1251

In [694]:
#we voegen lidmaatschap_actief kolom toe aan df_account_ACTIEF waar allemaal = 1
df_account_ACTIEF['lidmaatschap_actief'] = 1
df_account_INACTIEF['lidmaatschap_actief'] = 0

C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3507007459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_ACTIEF['lidmaatschap_actief'] = 1
C:\Users\emmad\AppData\Local\Temp\ipykernel_8868\3507007459.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_account_INACTIEF['lidmaatschap_actief'] = 0


In [695]:
df_account_INACTIEF.shape

(4060, 15)

In [696]:
#we voegen de df account_ACTIEF toe aan df_account_INACTIEF en noemen het df
df = pd.concat([df_account_INACTIEF, df_account_ACTIEF], axis=0)
df.drop('accountStatus', axis=1, inplace=True)
df.drop('ondernemingsaard', axis=1, inplace=True)
df.shape

(6612, 13)

In [697]:
df.head()

,accountID,plaats,isVokaEntiteit,ondernemingstype,activiteitNaam,lidmaatschapID,redenAangroei,redenVerloop,startDatum,opzegDatum,voorbije_jaar,aantal_inschrijvingen,lidmaatschap_actief
0,00002DAC-0A69-E111-B43A-00505680000A,BEVERE,0,Bedrijf,Overige industrie & diensten,F10DF4A4-186F-E111-B43A-00505680000A,unknown,unknown,2011-12-07,2013-12-31,2012,0.0,0
2,0012D444-0A69-E111-B43A-00505680000A,GERAARDSBERGEN,0,Bedrijf,None,DFECE67E-146F-E111-B43A-00505680000A,unknown,Geen gebruik,2012-03-02,2016-03-28,2015,0.0,0
7,003B965B-C268-E111-B43A-00505680000A,GENT,0,Bedrijf,Telecom & IT,D7D751D6-2D6F-E111-B43A-00505680000A,Actieve werving: koude prospectie,Geen gebruik,2013-07-04,2017-01-19,2016,0.0,0
12,005C6F7F-A268-E111-B43A-00505680000A,SINT-MARTENS-LATEM,0,Bedrijf,None,4F8AA590-BA47-EC11-8C62-6045BD8D2834,Lead van een collega nalv contact of deelname,Prijs,2021-11-16,2021-12-30,2020,0.0,0
13,0062F706-0969-E111-B43A-00505680000A,DEINZE,0,Bedrijf,None,45C679CB-086F-E111-B43A-00505680000A,unknown,Geen gebruik,2007-01-02,2013-07-01,2012,0.0,0


In [698]:
# welke verschillende mogelijkheden binnen 'ondernemingsaard' zijn er? geef elke mogelijkheid in lijst
df['activiteitNaam'].unique()

array(['Overige industrie & diensten', None, 'Telecom & IT',
       'Distributie, logistiek en transport', 'Consultancy', 'Energie',
       'Grafische industrie en diensten', 'Voeding', 'Detailhandel',
       'Overige industrie & diensten,Horeca & toerisme', 'Ijzer en staal',
       'Chemie, petrochemie', 'Textiel, kleding en confectie',
       'Financiële diensten', 'Horeca & toerisme', 'Papier & karton',
       'Verzekering', 'Human capital', 'Groothandel,Chemie, petrochemie',
       'Vastgoed', 'Zorg', 'Bouw', 'Accountancy & boekhouding',
       'Agrarische & bio-industrie', 'Telecom & IT,Papier & karton',
       'Telecom & IT,Consultancy', 'Vrije beroepen',
       'Technologische industrie & diensten,Grafische industrie en diensten',
       'Hout- en meubelindustrie', 'Bouw,Grafische industrie en diensten',
       'Distributie, logistiek en transport,Glas, rubber en kunststof',
       'Technologische industrie & diensten', 'Groothandel',
       'Technologische industrie & diensten,

## 3. Data Cleaning

Nu gaan we voor deze gemergede dataframe nog onehotencoding toepassen 

In [699]:
#one hot encoding voor df_account_INACTIEF 
#USING sklearn OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

ondernemingstype_cat = ['Bedrijf', 'Familiebedrijf', 'Sociale organisatie',
       'Vrije beroepen', 'Social Profit', 'Pers/Media', 'Onderwijs',
       'unknown', 'Overheid', 'Eenmanszaak', 'Werkgeversorganisaties',
       'Multinational', 'Beroepsorganisatie']

activiteitnaam_cat = ['Overige industrie & diensten', None, 'Telecom & IT',
       'Distributie, logistiek en transport', 'Consultancy', 'Energie',
       'Grafische industrie en diensten', 'Voeding', 'Detailhandel',
       'Overige industrie & diensten,Horeca & toerisme', 'Ijzer en staal',
       'Chemie, petrochemie', 'Textiel, kleding en confectie',
       'Financiële diensten', 'Horeca & toerisme', 'Papier & karton',
       'Verzekering', 'Human capital', 'Groothandel,Chemie, petrochemie',
       'Vastgoed', 'Zorg', 'Bouw', 'Accountancy & boekhouding',
       'Agrarische & bio-industrie', 'Telecom & IT,Papier & karton',
       'Telecom & IT,Consultancy', 'Vrije beroepen',
       'Technologische industrie & diensten,Grafische industrie en diensten',
       'Hout- en meubelindustrie', 'Bouw,Grafische industrie en diensten',
       'Distributie, logistiek en transport,Glas, rubber en kunststof',
       'Technologische industrie & diensten', 'Groothandel',
       'Technologische industrie & diensten,Bouw',
       'Automobiel- en Tweewielerindustrie',
       'Telecom & IT,Overige industrie & diensten,Vastgoed',
       'Overige industrie & diensten,Onderwijs', 'Havengerelateerd',
       'Media', 'Onderwijs', 'Overige industrie & diensten,Bouw',
       'Overheid',
       'Overige industrie & diensten,Automobiel- en Tweewielerindustrie',
       'Textiel, kleding en confectie,Detailhandel',
       'Milieu,Overige industrie & diensten,Energie',
       'Voeding,Overige industrie & diensten',
       'Zorg,Overige industrie & diensten',
       'Groothandel,Overige industrie & diensten',
       'Textiel, kleding en confectie,Groothandel,Detailhandel,Onderwijs',
       'Milieu',
       'Technologische industrie & diensten,Agrarische & bio-industrie',
       'Groothandel,Energie',
       'Ijzer en staal,Overige industrie & diensten',
       'Human capital,Technologische industrie & diensten,Zorg,Chemie, petrochemie,Consultancy',
       'Telecom & IT,Grafische industrie en diensten',
       'Overige industrie & diensten,Detailhandel',
       'Vastgoed,Verenigingen en maatschappelijke organisaties',
       'Verenigingen en maatschappelijke organisaties',
       'Textiel, kleding en confectie,Bouw,Horeca & toerisme',
       'Overige industrie & diensten,Chemie, petrochemie,Energie',
       'Milieu,Energie', 'Farmacie',
       'Technologische industrie & diensten,Overige industrie & diensten',
       'Overheid,Onderwijs',
       'Textiel, kleding en confectie,Technologische industrie & diensten',
       'Voeding,Chemie, petrochemie',
       'Technologische industrie & diensten,Overige industrie & diensten,Consultancy']
redenAangroei_cat = ['unknown', 'Actieve werving: koude prospectie',
       'Lead van een collega nalv contact of deelname',
       'Overzetting lidmaatschap', 'Spontane aanvraag via de website',
       'Actieve werving: tip van een collega',
       'Actieve werving: marketingcampagne',
       'Lead van een andere Voka entiteit',
       'Leden maken leden: Lead van een Voka lid',
       'Spontane aanvraag via mail']
redenVerloop_cat = ['unknown', 'Geen gebruik', 'Prijs', 'Overname', 'Stopzetting',
       'Wanbetaler', 'Ontevreden/klacht', 'Overzetting lidmaatschap',
       'Geen meerwaarde', 'Faillissement', 'FinanciÃ«le moeilijkheden']


In [700]:
activiteitNaam_encoder = OneHotEncoder(categories=[activiteitnaam_cat], sparse=False)
ondernemingstype_encoder = OneHotEncoder(categories=[ondernemingstype_cat], sparse=False)
redenAangroei_encoder = OneHotEncoder(categories=[redenAangroei_cat], sparse=False)
redenVerloop_encoder = OneHotEncoder(categories=[redenVerloop_cat], sparse=False)

activiteitNaam_1hot = activiteitNaam_encoder.fit_transform(df[['activiteitNaam']])
ondernemingstype_1hot = ondernemingstype_encoder.fit_transform(df[['ondernemingstype']])
redenAangroei_1hot = redenAangroei_encoder.fit_transform(df[['redenAangroei']])
redenVerloop_1hot = redenVerloop_encoder.fit_transform(df[['redenVerloop']])

c:\Users\emmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\emmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\emmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\emmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocess

In [701]:
# # # Ondernemingstype
# ondernemingstype_categories = [
#  {'categorie': 'unknown', 'binary': None},
#  {'categorie': 'Beroepsorganisatie', 'binary': None},
#  {'categorie': 'Vakbonden', 'binary': None},
#  {'categorie': 'Eenmanszaak', 'binary': None},
#  {'categorie': 'Multinational', 'binary': None},
#  {'categorie': 'Sociale organisatie', 'binary': None},
#  {'categorie': 'Werkgeversorganisaties', 'binary': None},
#  {'categorie': 'Pers/Media', 'binary': None},
#  {'categorie': 'Overheid', 'binary': None},
#  {'categorie': 'Onderwijs', 'binary': None},
#  {'categorie': 'Social Profit', 'binary': None},
#  {'categorie': 'Vrije beroepen', 'binary': None},
#  {'categorie': 'Familiebedrijf', 'binary': None},
#  {'categorie': 'Bedrijf', 'binary': None}
# ]

# for i, categorie in enumerate(ondernemingstype_categories):
#   categorie['binary'] = str(bin(i)[2:].zfill(4))

# category_to_binary = {categorie['categorie']: categorie['binary'] for categorie in ondernemingstype_categories}
# df['ondernemingstype'] = df['ondernemingstype'].map(category_to_binary)

# for i in range(1, 5):
#     df[f'ondernemingstype_{i}'] = df['ondernemingstype'].apply(lambda x: int(str(x)[i-1]))

In [702]:
df = df.drop(['activiteitNaam', 'redenAangroei', 'redenVerloop', 'ondernemingstype' ], axis=1)

In [703]:
# df = df.join(pd.DataFrame(activiteitNaam_1hot), rsuffix='_activiteitNaam')
df = df.join(pd.DataFrame(redenAangroei_1hot), rsuffix='_redenAangroei')
df = df.join(pd.DataFrame(redenVerloop_1hot), rsuffix='_redenVerloop')

In [706]:
df.head()

,accountID,plaats,isVokaEntiteit,lidmaatschapID,startDatum,opzegDatum,voorbije_jaar,aantal_inschrijvingen,lidmaatschap_actief,0,...,1_redenVerloop,2_redenVerloop,3_redenVerloop,4_redenVerloop,5_redenVerloop,6_redenVerloop,7_redenVerloop,8_redenVerloop,9_redenVerloop,10
0,00002DAC-0A69-E111-B43A-00505680000A,BEVERE,0,F10DF4A4-186F-E111-B43A-00505680000A,2011-12-07,2013-12-31,2012,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0012D444-0A69-E111-B43A-00505680000A,GERAARDSBERGEN,0,DFECE67E-146F-E111-B43A-00505680000A,2012-03-02,2016-03-28,2015,0.0,0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,003B965B-C268-E111-B43A-00505680000A,GENT,0,D7D751D6-2D6F-E111-B43A-00505680000A,2013-07-04,2017-01-19,2016,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,005C6F7F-A268-E111-B43A-00505680000A,SINT-MARTENS-LATEM,0,4F8AA590-BA47-EC11-8C62-6045BD8D2834,2021-11-16,2021-12-30,2020,0.0,0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0062F706-0969-E111-B43A-00505680000A,DEINZE,0,45C679CB-086F-E111-B43A-00505680000A,2007-01-02,2013-07-01,2012,0.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Model trainen

Nu gaan we voor deze situatie een model trainen die op basis van de aantal_inschrijvingen in het voorbije jaar gaat voorspellen of een account zijn lidmaatschap gaat opzeggen of niet. We gaan dit doen voor de accounts die hun lidmaatschap hebben opgezegd bij Voka.

In [156]:
#Nu gaan we voor deze situatie een model trainen die op basis van de aantal_inschrijvingen in het voorbije jaar gaat voorspellen of een account in het volgende jaar het lidmaatchap gaat opzeggen
#We gaan hiervoor een random forest model gebruiken
#We gaan eerst de data opsplitsen in een train en test set
#We gaan de data opsplitsen in een train en test set



In [359]:
#opsplitsen in X en y
X = df.drop('lidmaatschap_actief', axis=1)
y = df['lidmaatschap_actief']

In [360]:
#opslitsen in train en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((5289, 14), (1323, 14), (5289,), (1323,))